In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
!pip install helper

In [0]:
from torch import nn
from torchvision import datasets,transforms
from torch import optim
import torch.nn.functional as F

In [0]:
# loading mnist dataset from torchvision module

# Defining a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),])

# Download and Load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/',download=True,train=True,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True)


In [0]:
def create_train_nn(dataset):
  # Creating a dense neural network with 2 hidden layers to classify handwritten digits
  model = nn.Sequential(nn.Linear(784,128),
                       nn.ReLU(),
                       nn.Linear(128,64),
                       nn.ReLU(),
                       nn.Linear(64,10),
                       nn.LogSoftmax(dim=1))
  
  # creterion variable contains the loss function 
  criterion = nn.NLLLoss()
  # optimizer will update the weights based on the gradient calculated using backpropagation
  optimizer = optim.SGD(model.parameters(),lr=0.003)
  
  epochs = 5
  for e in range(epochs):
    running_loss = 0
    for images,labels in dataset:
      
      # Converting 28*28 pixel image to 784 pixel vector
      images = images.view(images.shape[0],-1)
      
      # Training Pass
      output = model.forward(images)
      loss = criterion(output,labels)
      loss.backward()
      optimizer.step()
      
      # Zero all the previous gradients before next pass
      optimizer.zero_grad()
      running_loss += loss.item()
      
    else:
        print(f"Training loss : {running_loss/len(trainloader)}")
  
  return model

In [0]:
%matplotlib inline
import helper

images,labels = next(iter(trainloader))
img = images[0].view(1,784)

model = create_train_nn(trainloader)
#Turn off gradient to speed up this part
with torch.no_grad():
  logits = model.forward(img)
  
ps = F.softmax(logits,dim=1)
helper.view_classify(img.view(1,28,28),ps)

Training loss : 1.8918549244337752
Training loss : 0.8288188673603509
Training loss : 0.5181742148326912
Training loss : 0.4304565435914851
Training loss : 0.38638079650938384


AttributeError: ignored